In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('sentiment_reviews.csv')

#Clean the Data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['review'] = df['review'].apply(clean_text)

#Encode the Words in the Review
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])

#Encode the Labels
labels = df['label'].values

#Outlier Removal
# Removing reviews that are too short (e.g., less than 3 words)
filtered_sequences = [seq for seq in sequences if len(seq) >= 3]
filtered_labels = [labels[i] for i in range(len(labels)) if len(sequences[i]) >= 3]

#Pad/Truncate Remaining Data
max_length = 100  # Set maximum length for padding
X = pad_sequences(filtered_sequences, maxlen=max_length)
y = np.array(filtered_labels)

#Split the Data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#Obtain Batches of Training Data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

#Define the Network Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


#Model Training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

#Evaluate
loss, accuracy = model.evaluate(test_dataset)
print(f'Accuracy: {accuracy}')

#Predict on test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

#Classification report
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


861/861 ━━━━━━━━━━━━━━━━━━━━ 110s 122ms/step - accuracy: 0.5974 - loss: 0.6559 - val_accuracy: 0.6858 - val_loss: 0.5785
Epoch 2/5
861/861 ━━━━━━━━━━━━━━━━━━━━ 142s 123ms/step - accuracy: 0.7372 - loss: 0.5151 - val_accuracy: 0.6974 - val_loss: 0.5633
Epoch 3/5
861/861 ━━━━━━━━━━━━━━━━━━━━ 144s 125ms/step - accuracy: 0.7886 - loss: 0.4334 - val_accuracy: 0.7014 - val_loss: 0.5754
Epoch 4/5
861/861 ━━━━━━━━━━━━━━━━━━━━ 105s 122ms/step - accuracy: 0.8171 - loss: 0.3770 - val_accuracy: 0.6970 - val_loss: 0.6309
Epoch 5/5
861/861 ━━━━━━━━━━━━━━━━━━━━ 148s 128ms/step - accuracy: 0.8441 - loss: 0.3286 - val_accuracy: 0.7002 - val_loss: 0.7051
185/185 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.6947 - loss: 0.7089
Accuracy: 0.6943549513816833
185/185 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step
              precision    recall  f1-score   support

    Negative       0.68      0.73      0.70      2906
    Positive       0.71      0.66      0.69      2993

    accuracy                           0.69

In [3]:
model.save('sentiment_model.h5')